# Nurse Scheduling

Hospital administrators must schedule nurses so that the hospital’s patients are provided with adequate care. At the same time, in the face of tighter competition in the health care industry, they must pay careful attention to keeping costs down. 

From historical records, administrators estimated the minimum number of nurses to have on hand for the various times of the day and days of the week, as shown in the following table. 

| Shift | Time | Minimum number of nurses needed |
|--|--|--|
| 1 | Midnight-4am | 5 |
| 2 | 4am-8am | 12 |
| 3 | 8am-noon | 14 |
| 4 | noon-4pm | 8 |
| 5 | 4pm-8pm | 14 |
| 6 | 8pm-Midnight | 10 |

Nurses work 8 hours a day in two consecutive shifts. As a result, in each shift, there are two types of nurses: those that started in the previous shift (and are now working their second shift), and those that just started in this shift (and will be working in the next shift as well). 

**A) (English Summary)** Summarize the decision, the objective and the constraints in English. 

**Decision :**

schedule nurses for each shift

**Objective:**

Minimize the total cost 

**Constraints:**

- Minimun demnad for each shift

- Nurses's work hours <= 8

- Nurses work for consecutive 2 shifts


**B) (Concrete Formulation)** Translate the above into a concrete formulation, by writing the decision variables, objective, and the constraints.

**Decision Varibles:**
$$\begin{aligned}
X_{1},X_{2},X_{3},X_{4},X_{5},X_{6}: \text{minimium # of nurses needed for shift 1-6}\\
\end{aligned}$$
**Objective:**
$$\text{Minimize:} X_{1}+ X_{2} + X_{3}+ X_{4} + X_{5} + X_{6}$$
**Constraints:**
$$\begin{aligned}
\text{r1:} && X_{1}+ X_{6} &\ge 5\\
\text{r2:} && X_{1}+ X_{2} &\ge 12\\
\text{r3:} && X_{2}+ X_{3} &\ge 14\\
\text{r4:} && X_{3}+ X_{4} &\ge 8\\
\text{r5:} && X_{4}+ X_{5}  &\ge 14\\
\text{r6:} && X_{5}+ X_{6} &\ge 10\\
\end{aligned}$$

**C) (Abstract Formulation)** Generalize the concrete formulation above to be able to handle arbitrary number of shifts in a cycle and arbitrary number of nurses needed for a shift (assuming still that each nurse stays for 2 shifts and the last shift connects to the first shift). You must render the math nicely using LaTex. 

**Decision Varibles:**
$$\begin{aligned}
\text{}I \text{ be the set of number of shift} (I ={1,2,3,...,I})\\
\text{}X_{i}\text{ be #of nurses added in shift} i\\
\text{}q_{i} \text{ be the minimum #of nurses needed for each shift}i\\
\end{aligned}$$
**Objective:**
$$\text{Minimize:} \sum_{i\in I}X_{i}$$
**Constraints:**
$$\begin{aligned}
\text{r1:} && X_{i}+ X_{i+1} &\ge q_{i+1}, \text{when i = 1 to I-1}\\
\text{r2:} && X_{I}+ X_{1} &\ge q_{1}\\
\end{aligned}$$

**D) (Numerical Solution)** Use Gurobi to solve the abstract formulation in part C) using the data above.


In [1]:
import pandas as pd
import numpy as np
import gurobipy as grb
import math
from gurobipy import Model, GRB

In [87]:
schedule = pd.read_excel('hw08.xlsx', sheet_name = 'Nurse')

In [88]:
schedule

,Time,Nurses
1,Midnight-4am,5
2,4am-8am,12
3,8am-noon,14
4,noon-4pm,8
5,4pm-8pm,14
6,8pm-Midnight,10


In [91]:
#D Abstract solution
mod = grb.Model()

I = schedule.index
qi = schedule.Nurses

X = mod.addVars(I,vtype = GRB.INTEGER)

mod.setObjective(sum(X[i+1] for i in range(len(I)) ))

for i in range(len(I)-1):
    mod.addConstr(X[i+1] + X[i+2] >= qi.loc[i+2] )

mod.addConstr(X[len(I)] + X[1] >= qi.loc[1])

mod.optimize()

Optimize a model with 6 rows, 6 columns and 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+01]
Found heuristic solution: objective 33.0000000
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        33.00000   33.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 33 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.300000000000e+01, best bound 3.300000000000e+01, gap 0.0000%


In [92]:
print('Minimum total # of nurses:',mod.objval)
for var in mod.getVars():
    print (var.x)

Minimum total # of nurses: 33.0
0.0
12.0
2.0
6.0
8.0
5.0



 # Project Selection
Ebony is an ambitious master's student who would like to maximize the number of extra-curricular business analytics projects she takes part of this year. However, projects may conflict with one another. The following graph summarizes the conflicts. (For example, project A conflicts with B, C and D, but projects B and D can be done together.)

<img src='Hwk8-Ebony.png'>

Beside the conflict above, 

- Project A is a prerequisite to project F (meaning that pursuing F requires also pursuing A.)
- Project B is a prerequisite to project G.

Formulate an optimization problem to help her decide which projects to pursue.

**A) (English Summary)** Summarize the decision, objective and constraints in English.

*Decision :**

decide whether to do project A to G

**Objective:**

aximize the number of extra-curricular business analytics projects 

**Constraints:**

- conclict given by the graph

- must done A before F

- must done B before G

**B) (Concrete Formulation)** Using the data above, translate the english summary into a concrete formulation of a LP or MIP, by giving the decision variables, objective, and the constraints.

**Decision Varibles:**
$$\begin{aligned}
X_{1},X_{2},X_{3},X_{4},X_{5},X_{6},X_{7}: \text{binary variables, whether to do project A to G}\\
\end{aligned}$$
**Objective:**
$$\text{Maximize:} X_{1}+ X_{2} + X_{3}+ X_{4} + X_{5} + X_{6}+ X_{7}$$
**Constraints:**
$$\begin{aligned}
\text{r1:} && X_{1} - X_{2} &\ge 0\\
\text{r2:} && X_{2} - X_{7} &\ge 0\\
\text{r3:} && X_{1}+ X_{2} &\le 1\\
\text{r4:} && X_{1}+ X_{3} &\le 1\\
\text{r5:} && X_{1}+ X_{4} &\le 1\\
\text{r6:} && X_{2}+ X_{3} &\le 1\\
\text{r7:} && X_{4}+ X_{5} &\le 1\\
\text{r8:} && X_{5}+ X_{6} &\le 1\\
\text{r9:} && X_{5}+ X_{7} &\le 1\\
\text{r10:} && X_{6}+ X_{7} &\le 1\\
\end{aligned}$$

**C) (Abstract Formulation)** Write an abstract formulation that generalizes the above to arbitrary datasets. You should define data variables to encode all the input data needed.

**Decision Varibles:**
$$\begin{aligned}
\text{let }I \text{ be the set of number of project} (I ={0,1,2,3,...,I-1})\\
i,j: i,j\in I\\
X_{i}: \text{whether to do project i}\\
X_{j}: \text{whether to do project j }
\end{aligned}$$
**Objective:**
$$\text{Maximize:} \sum_{i\in I}X_{i}$$
**Constraints:**
$$\begin{aligned}
\text{r1:} && X_{i}+ X_{j} &\le 1\\
\text{r2:} && X_{0}+ X_{5} &\ge 0\\
\text{r3:} && X_{1}+ X_{6} &\ge 0\\
\end{aligned}$$

**D) (Numerical Solution)** Use Gurobi to solve the abstract formulation from part C) using the data above.

In [154]:
# 1 means no conflict/ 0 otherwise
# 0 to 6 denote project A to G
project = pd.read_excel('hw08.xlsx', sheet_name = 'project')
project.drop(['project'],inplace = True, axis=1)
project

,0,1,2,3,4,5,6
0,1,0,0,0,1,1,1
1,0,1,0,1,1,1,1
2,0,0,1,1,1,1,1
3,0,1,1,1,0,1,1
4,1,1,1,0,1,0,0
5,1,1,1,1,0,1,0
6,1,1,1,1,0,0,1


In [158]:
mod = grb.Model()

I = project.index
X = mod.addVars(I,vtype = GRB.BINARY, )

mod.setObjective(sum(X[i] for i in range(len(I))),sense=GRB.MAXIMIZE)


for i in range(len(project.index)):
    for j in range(i+1):
        if  project.iloc[i].iloc[j] == 0:
            mod.addConstr(X[i] + X[j] <= 1)

mod.addConstr(X[0]-X[5] >= 0)            
mod.addConstr(X[1]-X[6] >= 0)  

mod.optimize()      

Optimize a model with 10 rows, 7 columns and 20 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 10 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [168]:
print('Optimal list of projects to pursue:',mod.objval)
a=[] #binary outcomes list
for var in mod.getVars():
    a.append(var.x)

p = ["A","B","C","D","E","F","G"]

for i in range(len(a)):
    if a[i] == 1:
        print(p[i])

Optimal list of projects to pursue: 3.0
B
D
G


In [161]:
print('Optimal list of projects to pursue:', "B D G")

Optimal list of projects to pursue: B D G


# Team Assignment

The following MIP is a simplified version of the program used to assign students into project teams in this course, to balance the overall characteristics of each team.

**Data:**

- $I$: set of students.
- $n$: number of teams
- $J=\{1,2,\cdots,n\}$ : set of teams.
- $K$: set of characteristics.
- $a_{ik}$: student $i$'s value for characteristics $k$.
- $w_k$: the weight for characteristics $k$ in the objective.
- $L_k$: the best lower bound for the sum of characteristic $k$ for any team. 
- $U_k$: the best upper bound for the sum of characteristics $k$ for any team.

You should assume that the data is given in a excel file with the same format as the `Q6-input-1.xlsx` and `Q6-input-2.xlsx` files attached to this assignment. 

The sheet named "Students" encodes $I$, $K$ and $a_{ik}$'s. In the below screenshot of `Q6-input-1.xlsx`, $I=\{A,B,C,D,E,F\}$, and $K=\{Person, Male, Programmer, Math, Speaking\}$.

<img src='Hwk8-Q6-1a.png' width=500>

The sheet named "Parameters" encodes the $w_k$, $L_k$ and $U_k$ for each characteristic $k$.

<img src='Hwk8-Q6-1b.png' width=500>

**Decision variables:**

- $x_{ij}$ : whether to assign student $i$ to team $j$. (Binary)
- $s_k$,  and $t_k$ : auxilliary variables for each characteristic $k$. (Continuous)

**MIP:**

$$\begin{aligned}
\text{Minimize:} && \sum_{k \in K} w_k(s_k+t_k) \\
\text{subject to:} && \\
\text{(Every person assigned)} && \sum_{j \in J} x_{ij} & = 1 && \text{For each person $i \in I$.}\\
\text{(Team balance)} && L_k - s_k \le \sum_{i \in I} a_{ik}x_{ij} & \le U_k + t_k && \text{For each team $j \in J$ and each $k \in K$.} \\
\text{(Binary)} && x_{ij} & \in \{0, 1\} && \text{for all $i$ and $j$.}\\
\text{(Non-negativity)} && s_k, t_k & \ge 0 && \text{for all $k$.}
\end{aligned}$$

**Write a function called "assignTeams" with the following input arguments**:

- **inputFile:** path to the input spreadsheet.
- **n:** the number of teams to divide students into.

**The function should return two variables:**

- **df:** a DataFrame with one column called "Team". The index should be the name of each individual, and the column "Team" should specify the number $j$ to which the person is assigned.
- **objval:** the optimal objective value.

In [240]:
# original code
students = pd.read_excel('Q6-input-1.xlsx', sheet_name = 'Students', index_col = 0 )
para = pd.read_excel('Q6-input-1.xlsx', sheet_name = 'Parameters')

n = 2

mod = grb.Model()

I = students.index
K = students.columns
w = para.iloc[0]

J = []
for i in range(n):
    J.append(i+1)

x = mod.addVars(I,J, vtype = GRB.BINARY)
s= mod.addVars(K)
t= mod.addVars(K)

mod.setObjective(sum(w[k] * (s[k] + t[k]) for k in K))

for i in I:
    mod.addConstr(sum(x[i,j] for j in J) == 1)

for k in K:
    for j in J:
        mod.addConstr(sum(x[i,j] * students.loc[i,k] for i in I) <= para.loc["U"][k] + t[k])
        mod.addConstr(sum(x[i,j] * students.loc[i,k] for i in I) >= para.loc["L"][k] - s[k])

mod.optimize()

Optimize a model with 26 rows, 22 columns and 104 nonzeros
Variable types: 10 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 24.0000000
Presolve removed 16 rows and 11 columns
Presolve time: 0.01s
Presolved: 10 rows, 11 columns, 46 nonzeros
Variable types: 0 continuous, 11 integer (9 binary)

Root relaxation: objective 0.000000e+00, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 0 nodes (5 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 0 24 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.00000

In [6]:
#function

def assignTeams(inputFile,n):
    
    students = pd.read_excel(inputFile, sheet_name = 'Students', index_col = 0 )
    para = pd.read_excel(inputFile, sheet_name = 'Parameters')
    
    mod = grb.Model()

    I = students.index
    K = students.columns
    w = para.iloc[0]

    J = []
    for i in range(n):
        J.append(i+1)

    x = mod.addVars(I,J, vtype = GRB.BINARY)
    s= mod.addVars(K)
    t= mod.addVars(K)

    mod.setObjective(sum(w[k] * (s[k] + t[k]) for k in K))

    for i in I:
        mod.addConstr(sum(x[i,j] for j in J) == 1)

    for k in K:
        for j in J:
            mod.addConstr(sum(x[i,j] * students.loc[i,k] for i in I) <= para.loc["U"][k] + t[k])
            mod.addConstr(sum(x[i,j] * students.loc[i,k] for i in I) >= para.loc["L"][k] - s[k])

    mod.optimize()
    
    ans= pd.DataFrame(index=I,columns=["Team"])
    
    for i in I:
        for j in J:
            if x[i,j].x:
                ans.loc[i,"Team"]=j
    return(ans, mod.objval)


In [7]:
df,objval=assignTeams('Q6-input-1.xlsx',2)
print('Optimal objective value',objval)
df

Optimize a model with 26 rows, 22 columns and 104 nonzeros
Variable types: 10 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 24.0000000
Presolve removed 16 rows and 11 columns
Presolve time: 0.00s
Presolved: 10 rows, 11 columns, 46 nonzeros
Variable types: 0 continuous, 11 integer (9 binary)

Root relaxation: objective 0.000000e+00, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 0 nodes (5 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 0 24 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.00000

,Team
Names,
A,2
B,2
C,2
D,1
E,1
F,1


In [8]:
df,objval=assignTeams('Q6-input-2.xlsx',10)
print('Optimal objective value',objval)
df.sort_values(by='Team')

Optimize a model with 151 rows, 520 columns and 3150 nonzeros
Variable types: 10 continuous, 510 integer (510 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 83.0000000
Presolve time: 0.01s
Presolved: 151 rows, 520 columns, 3150 nonzeros
Variable types: 0 continuous, 520 integer (512 binary)

Root relaxation: objective 1.100000e+00, 230 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.10000    0   41   83.00000    1.10000  98.7%     -    0s
H    0     0                      18.0000000    1.10000  93.9%     -    0s
H    0     0                       8.0000000    1.10000  86.2%     -    0s
     0     0     cutoff    0         8.00000    8.00000  0.00%     -    0s

Cutting planes:
  Gomory: 1

,Team
Names,
Bob,1
Yingying,1
Patty,1
I-Ting,1
Wayne,1
XingZhou,2
Shawn,2
Qinan,2
Mei,2
